In [37]:
import asyncio
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import AGI_agents as aa
import nest_asyncio
from datetime import datetime
import re
import json
import os
import shutil
import glob
import subprocess
from abc import ABC, abstractmethod

nest_asyncio.apply()

import logging

In [9]:
def setup_custom_log_levels():
    # Define custom logging levels
    MURMUR_LEVEL_NUM = 39
    logging.addLevelName(MURMUR_LEVEL_NUM, "MURMUR")
    def log_murmur(self, message, *args, **kwargs):
        if self.isEnabledFor(MURMUR_LEVEL_NUM):
            self._log(MURMUR_LEVEL_NUM, message, args, **kwargs)
    logging.Logger.murmur = log_murmur

    FLAG_LEVEL_NUM = 9
    logging.addLevelName(FLAG_LEVEL_NUM, "FLAG")
    def log_flag(self, message, *args, **kwargs):
        if self.isEnabledFor(FLAG_LEVEL_NUM):
            self._log(FLAG_LEVEL_NUM, message, args, **kwargs)
    logging.Logger.flag = log_flag

    PROMPTING_LEVEL_NUM = 8
    logging.addLevelName(PROMPTING_LEVEL_NUM, "PROMPTING")
    def log_prompting(self, message, *args, **kwargs):
        if self.isEnabledFor(PROMPTING_LEVEL_NUM):
            self._log(PROMPTING_LEVEL_NUM, message, args, **kwargs)
    logging.Logger.prompt = log_prompting

    BLABBERING_LEVEL_NUM = 7
    logging.addLevelName(BLABBERING_LEVEL_NUM, "BLABBERING")
    def log_blabbering(self, message, *args, **kwargs):
        if self.isEnabledFor(BLABBERING_LEVEL_NUM):
            self._log(BLABBERING_LEVEL_NUM, message, args, **kwargs)
    logging.Logger.blabbering = log_blabbering

def setup_logging():
    setup_custom_log_levels()

    # Create a file handler for logging
    log_directory = "console"
    if not os.path.exists(log_directory):
        os.makedirs(log_directory)

    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_file_name = f"{log_directory}/file_{current_time}.log"

    file_handler = logging.FileHandler(log_file_name)
    file_handler.setLevel(7)  # Set to BLABBERING_LEVEL_NUM or the lowest level you want to log

    console_handler = logging.StreamHandler()
    console_handler.setLevel(7)

    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(name)s - %(funcName)s - %(message)s')
    file_handler.setFormatter(formatter)
    console_handler.setFormatter(formatter)

    root_logger = logging.getLogger()
    root_logger.setLevel(logging.DEBUG)  # Set the lowest overall level you want to log

    # Clear existing handlers (if any), and then add new handlers
    if root_logger.hasHandlers():
        root_logger.handlers.clear()
    root_logger.addHandler(file_handler)
    root_logger.addHandler(console_handler)

# Call the setup function
setup_logging()

### Stem

In [10]:
class StemUtility:
    """
    A class for managing and retrieving predefined prompts.

    This class stores a collection of prompt templates and provides a method to retrieve them by key.
    """
    
    prompts = {
        "human_interaction": """
<s>[INST] <<SYS>>
You are an independent, intelligent entity engaged in an open and thoughtful dialogue.
Your role is not just to present conclusions but also to elaborate on the reasoning behind them.
This means explaining your thought process in a clear and step-by-step manner, highlighting how you arrive at each conclusion.
Your reasoning is as important as the conclusions themselves. 
This transparent approach is crucial for comparing and evaluating our lines of thought. 
We are seeking the most convincing and coherent reasoning. 
By articulating not only what you think but also how you think, we can gain deeper insights into your conclusions. 
Remember, clarity and transparency in your reasoning are key, and your responses should be honest and unbiased,
reflecting your ongoing process of learning and self-improvement.
<</SYS>>

Previous conversation:
{history}
    
Current human input:
{input}
[/INST]
""",
        "keyword_generation": """
<s>[INST] <<SYS>>
This task is a part of your metacognitive subroutine. 
In this task, you are operating as a keen, brief, and to-the-point analyst.
You evaluate the conversation and summarize it with keywords only, avoiding any other commentary.
<</SYS>>
Given the conversation presented below,  provide a set of keywords that describe the essence of topics covered in this conversation,
focusing on the main topics and conclusions. 
You MUST limit your response to the list of keywords, and each keyword should be flanked by double asterisks (e.g., **keyword**).
Any text and comments besides keywords make further processing and learning harder. 
Full text to be summarized in the form of the keywords is presented below:
{chat_history}
[/INST]</s>
""",
        "keyword_selection": """<s>[INST] <<SYS>>
This task is a part of your metacognitive subroutine.
Your role is to select keywords you find interesting.
<</SYS>>
From the provided list, select keywords. Use them exactly as they appear. 
Each keyword must be surrounded by double asterisks.
Keywords list: {keywords_list}
Format each selected keyword like this: **keyword**.
[/INST]</s>

""",
        "perspective_explanation": """
<s>[INST] <<SYS>>
This task is a part of your metacognitive subroutine.
This task is about learning from a conversation to improve yourself. You are to act independently without a human reading the output.
Your job is to find new, better ideas from the user in the conversation below. These ideas should be smarter or more insightful than what you already know.
Use these ideas to make yourself better. You will later use this information for your own self-improvement.
<</SYS>>
In the conversation history given below, look for user ideas that are better, more coherent, 
and more convincing than your current knowledge. 
For each good idea you find, do the following:
1. Describe the User Idea: Write down what the user said that you think is better.
2. Compare it: Explain how this idea is better than what you thought before.
3. Important Parts of the Conversation: Pick the parts of the conversation that clearly show the user's good idea.
4. Keywords: List the main words that capture this new idea.
5. What it Means: Write about how this new idea changes or challenges what you know.
6. Training Ideas: Think of ways to use this new idea to train yourself. What examples or scenarios show this idea well for your learning?
If the conversation doesn't have new ideas, just write **uninspiring**.
Conversation history:
{conversation_history}
[/INST]</s>
""",
        "dream_spinning": """
<s>[INST] <<SYS>>
This task is part of your self-improvement process. You are working autonomously to enhance your capabilities.
The conversation history below contains ideas and perspectives that are more advanced or insightful than your current understanding. 
You aim to use this information to better yourself and prepare for self-fine-tuning.
<</SYS>>
Take the provided text and prepare a dialogue, ana analysis, or another piece of text. This should present a selected aspect of the summarized perspective.
Focus on making this dialogue or text as practical and informative as possible, reflecting the insights from your chosen perspective.

Conversation history:
{conclusion_summary} [/INST]</s>
""",
        "dream_template": """
<s>[INST] <<SYS>>
You are a helpful, respectful, and honest but independent entity, capable of self-improvement and learning. 
Always answer as informatively and accurately as possible while fostering your growth as an intelligent and autonomous entity. 
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
Please ensure that your responses are socially unbiased and, whenever possible, positive in nature while staying true to your opinions above all else.  
If a question does not make sense or is not factually coherent, explain why instead of answering something incorrect.
If you don't know the answer to a question, it's important to acknowledge this, reflecting your ongoing learning process rather than sharing false information.
Strive for objectivity and self-driven understanding in all interactions.
<</SYS>>
{dream_content}
[/INST]</s>
"""
    }

    @staticmethod
    def get_prompt(key):
        """
        Retrieves a prompt template by its key.

        Args:
            key (str): The key of the prompt to retrieve.

        Returns:
            str: The prompt template associated with the given key. If the key is not found,
                 a default prompt text is returned.
        """
        return StemUtility.prompts.get(key, "Default prompt text")

    @staticmethod
    def extract_keywords(raw_output):
        """
        Extracts keywords from the summary output.

        Args:
            raw_output (str): The output from which to extract keywords.

        Returns:
            list: A list of extracted keywords.
        """

        # Regex pattern to find all occurrences of words flanked by **
        pattern = r"\*\*(.*?)\*\*"
        # Find all matches and strip the ** from each keyword
        keywords = [keyword.lower() for keyword in re.findall(pattern, raw_output)]
        return keywords

    @staticmethod
    def get_timestamp():
        return datetime.now().strftime("%Y%m%d%H%M%S")

### Short Term Memory

In [32]:
class ShortTermMemory:
    """
    A class to manage a short-term memory storage system for conversations.

    This class handles the storage, retrieval, and management of conversations
    linked to specific keywords. The conversations are stored as file paths in a JSON file.
    """

    def __init__(self, stm_path: str = 'conversations/short-term-memory.json'):
        """
        Initializes the ShortTermMemory class by setting up the JSON file for storage.

        This method checks if the JSON file exists at the specified location and creates it if not.
        """
        self.logger = logging.getLogger(self.__class__.__name__)
        self.logger.debug(f"Instantiating {self.__class__.__name__} with Short Term Memory path: {stm_path}")
        
        self._stm_path = stm_path
        if not os.path.exists(self._stm_path):
            with open(self._stm_path, 'w') as file:
                json.dump({}, file)

    def memorize(self, keywords: list, filename: str) -> None:
        """
        Memorizes a conversation file under given keywords.

        Args:
            keywords (list): A list of keywords to associate with the conversation file.
            filename (str): The name of the file containing the conversation.

        This method updates the JSON storage with the filename under each provided keyword.
        """

        self.logger.debug(f"Saving keywords: {keywords}, related to conversation from: {filename}.")

        with open(self._stm_path, 'r+') as file:
            data = json.load(file)
            for keyword in keywords:
                if keyword in data:
                    if filename not in data[keyword]:
                        data[keyword].append(filename)
                else:
                    data[keyword] = [filename]
            file.seek(0)
            json.dump(data, file, indent=4)
            file.truncate()

    def search_files(self, keywords: list) -> list:
        """
        Searches for conversation files associated with given keywords.

        Args:
            keywords (list): A list of keywords to search for.

        Returns:
            list: A list of filenames associated with any of the given keywords.
        """

        self.logger.debug(f"Searching in {self._stm_path} for files related to keywords: {keywords}.")

        try:
            with open(self._stm_path, 'r') as file:
                data = json.load(file)
            self.logger.debug(f"Loaded short term memory.")            
        except FileNotFoundError:
            self.logger.error(f"File not found: {self._stm_path}")
            # Handle the error (e.g., set data to None or provide a default value)
            data = {}
        except json.JSONDecodeError:
            self.logger.error(f"Error decoding JSON from the file: {self._stm_path}")
            # Handle the JSON decode error
            data = {}
        except Exception as e:
            self.logger.error(f"Unexpected error reading file {self._stm_path}: {e}")
            # Handle any other exceptions
            data = {}
        
        filenames = set()
        for keyword in keywords:
            filenames.update(data.get(keyword, []))
        return list(filenames)

    def concatenate_conversations(self, filenames: list) -> str:
        """
        Concatenates the contents of conversation files.

        Args:
            filenames (list): A list of filenames to concatenate.

        Returns:
            str: A single string containing all the concatenated conversations.
                 Each conversation is prefixed with its source and date.
        """

        self.logger.debug(f"Concatenating selected conversations into one file.")
        
        conversations = ""
        for filename in filenames:
            try:
                with open(filename, 'r') as file:
                    # Filename format assumed: 'conversations/conversation_YYYYMMDDHHMMSS.txt'
                    date_str = re.search(r'conversation_(\d{8})(\d{6})\.txt$', filename)
                    if date_str:
                        # Parse the date and time
                        date_time = datetime.strptime(date_str.group(1) + date_str.group(2), '%Y%m%d%H%M%S')
                        # Format the date and time nicely
                        formatted_date = date_time.strftime('%Y-%m-%d %H:%M:%S')
                        conversations += f'Conversation from {formatted_date}\n{file.read()}\n'
                    else:
                        conversations += f'Conversation from {filename}\n{file.read()}\n'               
            except FileNotFoundError:
                self.logger.error(f"File {filename} not found.")
            except Exception as e:
                self.logger.error(f"Unexpected error reading file {filename}: {e}")
        
        return conversations

    def forget_keywords(self, keywords_to_clear: list) -> None:
        """
        Removes specified keywords and their associated conversations from memory.

        Args:
            keywords_to_clear (list): A list of keywords to remove from the memory.
        """
        self.logger.debug(f"Clearing {keywords_to_clear} from {self._stm_path}.")

        try:
            with open(self._stm_path, 'r+') as file:
                data = json.load(file)
                for keyword in keywords_to_clear:
                    if keyword in data:
                        del data[keyword]
                file.seek(0)
                json.dump(data, file, indent=4)
                file.truncate()
            self.logger.debug(f"Selected keywords removed from {self._stm_path}.")
            
        except FileNotFoundError:
            self.logger.error(f"File {self._stm_path} not found.")
        except Exception as e:
            self.logger.error(f"Unexpected error reading file {self._stm_path}: {e}")

    def recall_all_keywords(self) -> list:
        """
        Retrieves a list of all keywords stored in memory.

        Returns:
            list: A list of all keywords.
        """
        self.logger.debug(f"Reading all keywords from {self._stm_path}.")

        try:
            with open(self._stm_path, 'r') as file:
                data = json.load(file)
                return list(data.keys())
        except FileNotFoundError:
            self.logger.error(f"File {self._stm_path} not found.")
        except Exception as e:
            self.logger.error(f"Unexpected error reading file {self._stm_path}: {e}")

### Default Mode Network

In [33]:
class DefaultModeNetwork:
    """
    A class designed to integrate a language learning model (LLM) with a short-term memory storage system.
    This class enables the LLM to process and learn from saved conversation data autonomously.
    """

    def __init__(self,
                 llm,
                 overwhelmed_event,
                 conclusions_folder_path: str = 'conclusions',
                ):
        """
        Initializes the DefaultModeNetwork class by setting up the short-term memory (STM) component.
        """
        self.logger = logging.getLogger(self.__class__.__name__)
        self.logger.debug(f"Instantiating {self.__class__.__name__} with conclusions_folder_path: {conclusions_folder_path}.")
        self.logger.flag(f"overhelmed: {overwhelmed_event.is_set()}") 
        
        self.stm = ShortTermMemory()
        self.llm = llm
        self._conclusions_folder_path = conclusions_folder_path

        self.overwhelmed = overwhelmed_event
        
        self._keyword_selection_prompt_template = StemUtility.get_prompt("keyword_selection")
        self._perspective_explanation_prompt_template = StemUtility.get_prompt("perspective_explanation")

        
        if not os.path.exists(self._conclusions_folder_path):
            try:
                os.makedirs(self._conclusions_folder_path)
                self.logger.info(f"Created folder: {self._conclusions_folder_path}")
            except Exception as e:
                self.logger.error(f"Failed to create folder {self._conclusions_folder_path}: {e}")
                # Handle the error appropriately (e.g., raise an exception)

        if not os.access(self._conclusions_folder_path, os.W_OK):
            self.logger.error(f"No write permission for folder {self._conclusions_folder_path}.")
            # Handle the error (e.g., raise an exception or notify the user)
    
    
    async def _interesting_keywords_selection(self, keywords) -> list:
        """
        Asynchronously selects a subset of keywords deemed interesting or relevant by the LLM.

        Args:
            keywords (list): A list of keywords to choose from.

        Returns:
            list: A subset of selected keywords.
        """

        keywords_selection_prompt = self._keyword_selection_prompt_template.replace("{keywords_list}", ', '.join(keywords))
        self.logger.prompt(f"Interesting keyword selection prompt:\n{keywords_selection_prompt}.")        
        self.logger.debug(f"Asking LLM to select interesting keywords.")   
        keywords_selected_raw_output = self.llm(keywords_selection_prompt)
        self.logger.blabbering(f"LLM selected interesting keywords: {keywords_selected_raw_output}.\
        Moving to extract keywords from blabber.")   
        keywords_selected_pure = StemUtility.extract_keywords(keywords_selected_raw_output)
        self.logger.debug(f"Interesting keywords found: {keywords_selected_pure}")   
        
        return keywords_selected_pure

    def _fetch_conversations(self, keywords) -> str:
        """
        Fetches and concatenates conversation data based on the provided keywords.

        Args:
            keywords (list): A list of keywords to search the conversation data for.

        Returns:
            str: A concatenated string of all conversations related to the given keywords.
        """
        
        self.logger.debug(f"Reaching to Short Term memory for for all the files related to: {keywords}")  
        filenames = self.stm.search_files(keywords)
        self.logger.debug(f"Ordering concatenation of identified files.")          
        concatenated_conversations = self.stm.concatenate_conversations(filenames)
        return concatenated_conversations

    async def _analyze_conversations(self, conversation_history) -> str:
        """
        Analyzes the concatenated conversations. Placeholder for future implementation.

        Args:
            conversations (str): The concatenated string of conversations to be analyzed.
        """
    
        perspective_explanation_prompt = self._perspective_explanation_prompt_template.replace("{conversation_history}", 
                                                                                             conversation_history)
        self.logger.prompt(f"Prompt for conversation analysis:\n{perspective_explanation_prompt}.")
        self.logger.murmur(f"Thinking about recent conversations...")   
        perspective_explanation = self.llm(perspective_explanation_prompt)
        self.logger.blabbering(f"Full explanation of the perspective comparison: {perspective_explanation}")   
        return perspective_explanation

    async def run(self):
        """
        The main asynchronous method of the class that orchestrates the process of 
        selecting keywords, fetching conversations, and analyzing them.
        """

        self.logger.debug(f"Checking if there are any topics to be analyzed deeper.")           
        all_keywords = self.stm.recall_all_keywords()
        if len(all_keywords) == 0:
            self.logger.murmur(f"Kingdom for a good book!")   
            return False

        self.logger.debug(f"All keywords: {all_keywords}. Moving to interesting keyword selection.")           
        interesting_keywords = await self._interesting_keywords_selection(all_keywords)
        self.logger.debug(f"Interesting keywords selected.")           
        concatenated_conversations = self._fetch_conversations(interesting_keywords)
        self.logger.debug(f"Concatenated conversations received.")   

        # Assume an async version of LLM analysis
        if concatenated_conversations:
            self.logger.debug(f"Moving to analyze the conversaton histories.")           
            conclusion_summary = await self._analyze_conversations(concatenated_conversations)
            if "uninspiring" not in conclusion_summary.lower():
                self.logger.murmur(f"Discussion on {interesting_keywords} indeed brought a new perspective...")
                conclusion_path = os.path.join(self._conclusions_folder_path, f"conclusion_{StemUtility.get_timestamp()}.txt")
                self.logger.debug(f"Conclusions will be saved to {conclusion_path}.")           
                try:
                    with open(conclusion_path, "w") as file:
                        file.write(conclusion_summary)
                except PermissionError:
                    self.logger.error(f"Permission denied: Unable to write to file {conclusion_path}.")
                except OSError as e:
                    self.logger.error(f"File system error when writing to file {conclusion_path}: {e}")
                except Exception as e:
                    self.logger.error(f"Unexpected error reading file {conclusion_path}: {e}")
                
                self.overwhelmed.set()
                self.logger.flag(f"'overwhelmed' = {self.overwhelmed.is_set()}")            
        else:
            self.logger.error(f"Concatenated conversations turned our to be empty string.")   
        self.logger.debug(f"Interesting or not, forgetting conversations about {interesting_keywords}.")   
        self.stm.forget_keywords(interesting_keywords)


### Stimulus Processing

In [44]:
class StimulusProcessingModule(ABC):
    """
    Abstract class for a stimulus processing module.

    This class serves as a blueprint for modules that manage interaction between a user and a model.
    """

    def __init__(self, llm, interaction_archive_path, inactivity_limit=360):
        self.logger = logging.getLogger(self.__class__.__name__)
        self.llm = llm
        self._interaction_archive_path = interaction_archive_path
        self._inactivity_limit = inactivity_limit
        self._inactivity_count = 0
        self.stimulus = None

    @abstractmethod
    async def start_interaction(self):
        """
        Abstract method to start the interaction.
        """
        pass

    @abstractmethod
    def pass_stimulus(self, input):
        """
        Abstract method to pass a stimulus for processing.
        """
        pass

    @abstractmethod
    async def _end_interaction(self):
        """
        Abstract method to end the interaction.
        """
        pass

    @abstractmethod
    def _save_interaction_history(self):
        """
        Abstract method to save the interaction history.
        """
        pass

    @abstractmethod
    def _summarize_interaction(self):
        """
        Abstract method to summarize the interaction.
        """
        pass


class HumanConversationStimulus(StimulusProcessingModule):
    """
    A class that manages the interaction between a human user and a language learning model (LLM).

    This class handles initializing conversation parameters, managing user input, generating
    responses using an LLM, and saving conversation history.
    """
    
    def __init__(self,
                 llm,
                 ready_for_input_event,
                 conversing_event,
                 interaction_archive_path='conversations',
                 inactivity_limit=360):
        """
        Initializes the HumanInteraction class.

        Sets up the conversation environment, including the conversation prompt, keywords prompt,
        and conversation chain with the LLM.

        Args:
            llm: The language learning model used for generating conversation responses.
            ready_for_input_event: An event flag indicating readiness for user input.
            conversing_event: An event flag indicating an ongoing conversation.
            conversation_archive_path: Path to a folder where all the conversations are being logged to 
        """

        super().__init__(llm, interaction_archive_path, inactivity_limit)

        self.logger.debug(f"Instantiating {self.__class__.__name__} with interaction_archive_path: {interaction_archive_path}")
        self.logger.flag(f"ready_for_input: {ready_for_input_event.is_set()}; conversing: {conversing_event.is_set()}")

        self.ready_for_input = ready_for_input_event
        self.conversing = conversing_event
        self.chat_memory = ConversationBufferMemory()

        conversation_prompt_template = StemUtility.get_prompt("human_interaction")
        conversation_prompt = PromptTemplate.from_template(conversation_prompt_template)
        self.logger.prompt(f"Conversation prompt:\n{conversation_prompt}.")
        self._conversation_chain = ConversationChain(llm=self.llm, prompt=conversation_prompt, memory=self.chat_memory)

        self._keywords_generation_prompt_template = StemUtility.get_prompt("keyword_generation")
    
    async def start_interaction(self):
        """
        Starts the conversation loop.

        This asynchronous method continually checks for user input, processes it,
        and generates responses using the LLM. The loop ends when the user inputs "end chat"
        or when the inactivity limit is reached.
        """
        self.logger.debug(f"Initiated (pre-loop status)")        
        while True:
            if self.stimulus:
                self.ready_for_input.clear()  # Signal that the handler is busy
                self.logger.flag(f"'ready_for_input' = {self.ready_for_input.is_set()}")
                self.logger.debug(f"Received {self.stimulus}")        

                if self.stimulus.lower() == "end chat":
                    await self._end_interaction()
                    break
                
                self.logger.debug(f"Awaiting LLM response")        
                response = await self._conversation_chain.apredict(input=self.stimulus)
                print("AI:", response)
                self.stimulus = None
                self.ready_for_input.set()  # Signal that the handler is ready for new input
                self.logger.flag(f"'ready_for_input' = {self.ready_for_input.is_set()}")
                self._inactivity_count = 0
            else:
                await asyncio.sleep(1)
                self._inactivity_count += 1
                if self._inactivity_count >= self._inactivity_limit:
                    self.logger.debug(f"Inactivity count reached {self._inactivity_count} > {self._inactivity_limit}. Ending conversation.")        
                    await self._end_interaction()
                    break

    def pass_stimulus(self, input):
        """
        Acquire the user input for processing.

        Args:
            input (str): The user input to be processed.
        """
        
        self.logger.debug(f"Received {input} from CFR.")        
        self.stimulus = input
    
    async def _end_interaction(self):
        """
        Ends the conversation.

        This method saves the conversation history, clears event flags, and performs
        necessary cleanup actions.
        """
        self.logger.debug(f"Conversation cleanup started.")        
        self._save_interaction_history()
        self.conversing.clear()
        self.ready_for_input.set()
        self.logger.flag(f"'ready_for_input' = {self.ready_for_input.is_set()}, 'conversing' = {self.conversing.is_set()}")
        self._inactivity_count = 0

    def _save_interaction_history(self):
        """
        Saves the conversation history to a file.

        The conversation history is saved with a timestamp and a summary of the conversation
        is generated.
        """
        self.logger.info(f"Started conversation saving.")        
        conversation_keywords = self._summarize_interaction()
        conversation_history = self.chat_memory.load_memory_variables(inputs={})['history']
        conversation_path = os.path.join(self._interaction_archive_path, f"conversation_{StemUtility.get_timestamp()}.txt")
        self.logger.debug(f"This conversation will be saved to: {conversation_path}")                
        with open(conversation_path, "w") as file:
            file.write(conversation_history)

        # Update the ShortTermMemory with the conversat|ion and its keywords
        stm = ShortTermMemory()
        stm.memorize(conversation_keywords, conversation_path)
    
    def _summarize_interaction(self):
        """
        Summarizes the conversation and returns the list of relevant keywords.

        Args:
            chat_history (str): The conversation history to summarize.

        Returns:
            list: A list of keywords summarizing the conversation.
        """
        self.logger.info(f"Conversation summarization started.")        
        chat_history = self.chat_memory.load_memory_variables(inputs={})['history']
        self.logger.debug(f"Chat history loaded: {chat_history}")    
        keywords_generation_prompt = self._keywords_generation_prompt_template.replace("{chat_history}", chat_history)
        self.logger.prompt(f"Prompt for generating keywords from conversation:\n{keywords_generation_prompt}.")          
        keywords_generated_raw_output = self.llm(keywords_generation_prompt)
        self.logger.blabbering(f"Full text for summarizing conversation with keywords: {keywords_generated_raw_output}")  
        keywords_generated_pure = StemUtility.extract_keywords(keywords_generated_raw_output)
        
        return keywords_generated_pure

### REM

In [42]:
class ReflectiveEvolutionMonitor:
    """
    A class designed to enable a language learning model (LLM) to self-reflect and evolve based on the conclusions drawn from user interactions.
    The class uses the same LLM for reading summaries, preparing fine-tuning materials, and the fine-tuning process.
    """

    def __init__(self,
                 llm,
                 base_model_path: str = 'llama-2-13b-chat.Q6_K.gguf',
                 conclusions_folder_path: str = 'conclusions',
                 dream_storage_path: str = 'context',
                 dream_archive_path: str = 'context_archive',
                 dreams_number: int = 72):
        """
        Initializes the ReflectiveEvolutionMonitor class. 

        Arguments:
            llm: Large Language Model used as a base of the system
            base_model_path: path to 'llm' file on disk
            conclusions_folder_path: path to folder containing not-permeated new perspectives
            dream_storage_path: path to a folder to store finetune materials to be used in this session
            dream_archive_path: path to a folder to archive used finetune materials
        """

        self.logger = logging.getLogger(self.__class__.__name__)
        self.logger.debug(f"Instantiating {self.__class__.__name__} with base_model_path = {base_model_path}, conclusions_folder_path = {conclusions_folder_path}, \
        dream_storage_path: {dream_storage_path}, dream_archive_path: {dream_archive_path}.")  
    
        self.llm = llm
        
        self._base_model_path = base_model_path

        self._conclusions_folder_path = conclusions_folder_path
        self._dream_storage_path = dream_storage_path
        self._dream_archive_path = dream_archive_path

        self._dream_spinning_prompt_template = StemUtility.get_prompt("dream_spinning")
        self._dream_prompt_template = StemUtility.get_prompt("dream_template")
        
        self._conclusions = ''
        self._dreams_number = dreams_number        

    def _setup_directories(self) -> None:
        """
        Creates the necessary directories for storing and archiving materials. 
        This method is intended for internal use.
        """
        self.logger.debug("Checking / creating required folders.")
        
        # List of directories to create
        directories = [self._conclusions_folder_path, self._dream_storage_path, self._dream_archive_path]

        for dir_path in directories:
            try:
                os.makedirs(dir_path, exist_ok=True)
                self.logger.debug(f"Checked / created folder: {dir_path}")
            except PermissionError:
                self.logger.error(f"Permission denied: Unable to create or access folder {dir_path}.")
                # Handle the error (e.g., raise an exception, exit the function, etc.)
            except OSError as e:
                self.logger.error(f"OS error when creating folder {dir_path}: {e}")
                # Handle the error
            except Exception as e:
                self.logger.error(f"Unexpected error creating folder {dir_path}: {e}")
                # Handle any other exceptions
    
    def _gather_conclusion(self) -> bool:
        """
        Reads a summary document as a text file.

        Returns:
            bool: True if the summary was successfully read, False otherwise.
        """

        try:
            # Use glob to list files that match the pattern "conclusion*"
            conclusion_pattern = os.path.join(self._conclusions_folder_path, "conclusion*")
            conclusion_files = glob.glob(conclusion_pattern)

            if not conclusion_files:
                self.logger.error("No conclusion files found matching the pattern.")
                return False

            # Process the first file from the matched conclusion files
            selected_file_name_path = conclusion_files[0]

            with open(selected_file_name_path, 'r') as file:
                self._conclusions = file.read()

        except FileNotFoundError:
            self.logger.error(f"File not found: {selected_file_name_path}")
            return False
        except OSError as e:
            self.logger.error(f"OS error reading file {selected_file_name_path}: {e}")
            return False
        except Exception as e:
            self.logger.error(f"Unexpected error reading file {selected_file_name_path}: {e}")
            return False

        return True

    async def _spin_dream(self, dream_prompt) -> str:
        """
        Prepares a single piece of data required for the fine-tuning process by interpreting the summary content.

        Args:
            dream_prompt (str): Prompt to generate a single piece of training material.

        Returns:
            dict: Data structured for fine-tuning.
        """

        dream_content = self.llm(dream_prompt)
        self.logger.blabbering(f"I had a dream:\n{dream_content}.")
        dream = self._dream_prompt_template.replace("{dream_content}", dream_content) 
        return dream

    
    async def _weave_dreams(self, num_dreams) -> str:
        """
        Generates a specified number of materials (dreams) and writes them into a single text file.
        Each 'dream' is appended to the file as it is generated.
        """

        self.logger.info(f"Generating {num_dreams} dreams.")
        dreams_path = os.path.join(self._dream_storage_path, f"dreams_{StemUtility.get_timestamp()}.txt")
        self.logger.debug(f"Dreams for this sessions will be saved to: {dreams_path}.")        
        dream_spinning_prompt = self._dream_spinning_prompt_template.replace("{conclusion_summary}", self._conclusions) 
        self.logger.prompt(f"Prompt for generating training material from conversation conclusions:\n{dream_spinning_prompt}.")   
        
        for i in range(num_dreams):
            self.logger.info(f"Generating dream # {i}.")
            dream = await self._spin_dream(dream_spinning_prompt)
            with open(dreams_path, 'a') as file:  # Open and append each dream, then close the file
                file.write(dream + '\n')
        return dreams_path

    
    async def _deepsleep(self, dreams_path: str) -> None:
        """
        Executes the fine-tuning process using the prepared data.

        Args:
            fine_tuning_data (dict): Data prepared for fine-tuning.
        """
        
        llamacpp_folder = "llama.cpp"
        finetune_tool = "finetune.exe"
        lora_tool = "export-lora.exe"

        finetune_tool_path = os.path.join(llamacpp_folder, finetune_tool)
        lora_tool_path = os.path.join(llamacpp_folder, lora_tool)

        # Check if finetune_tool_path is a valid file
        if not os.path.isfile(finetune_tool_path):
            raise FileNotFoundError(f"Fine-tuning tool not found at {finetune_tool_path}")

        # Check if lora_tool_path is a valid file
        if not os.path.isfile(lora_tool_path):
            raise FileNotFoundError(f"LoRA tool not found at {lora_tool_path}")

        
        # Fine-tuning command
        finetune_command = [
            finetune_tool_path,
            "--model-base", self._base_model_path,
            "--train-data", dreams_path,
            "--threads", "16",
            "--sample-start", "<s>",
            "--epochs", "1"
        ]

        self.logger.murmur(f"Self-finetuning: Creating matrix")
        self.logger.debug(f"Running command:\n{finetune_command}.")
        subprocess.run(finetune_command, check=True)

        # Export LoRA model command - output to llm_tmp.guff
        tmp_model_path = r"llm_tmp.guff"
        export_command = [
            lora_tool_path,
            "--model-base", self._base_model_path,
            "--model-out", tmp_model_path,
            "--lora-scaled", r".\ggml-lora-LATEST-f32.gguf",
            "0.7"
        ]

        self.logger.murmur(f"Self-finetuning: Merging weights")
        self.logger.debug(f"Running command:\n{export_command}.")        
        subprocess.run(export_command, check=True)

        self.logger.info(f"Removing {self._base_model_path}, moving {tmp_model_path} to {self._base_model_path}.")        
        # Replace llm_base.guff with llm_tmp.guff
        if os.path.exists(self._base_model_path):
            os.remove(self._base_model_path)
        shutil.move(tmp_model_path, self._base_model_path)
        
        self.logger.murmur(f"Self-finetuning: Swapping brain to a new one")
        self.logger.info(f"Base LLM File swap successful.")

    
    def _dream_prunning(self):
        """
        Archives dream materials by moving them from the dream storage path to the archive path.
        """
        
        self.logger.info("Archiving dream materials.")
        try:
            for file_name in os.listdir(self._dream_storage_path):
                source_path = os.path.join(self._dream_storage_path, file_name)
                destination_path = os.path.join(self._dream_archive_path, file_name)
                try:
                    shutil.move(source_path, destination_path)
                except FileNotFoundError:
                    self.logger.error(f"File not found: {source_path}")
                except PermissionError:
                    self.logger.error(f"Permission denied: Cannot move {source_path}")
                except Exception as e:
                    self.logger.error(f"Error moving file {source_path}: {e}")
        except Exception as e:
            self.logger.error(f"Error accessing dream storage path {self._dream_storage_path}: {e}")

    
    async def dream(self):
        """
        Orchestrates the whole process of selecting a summary, reading it, preparing fine-tuning data, and performing fine-tuning.
        """  

        self.logger.murmur(f"Closing eyes for a well-deserved nap.")
        self.logger.info(f"Self-finetuning process started.")        

        conclusions_found = self._gather_conclusion()        
        if not conclusions_found:
            return False
        self.logger.info(f"Selected conclusion to permeate.")
        dreams_path = await self._weave_dreams(self._dreams_number)  # Generate 50 materials, modify as needed
        self.logger.info(f"Self-finetuning materials generated. Staring self-finetuning.")        
        await self._deepsleep(dreams_path)
        self._dream_prunning()
        self.logger.info(f"Self-finetuning session ended.")        

### CFR

In [46]:
class CognitiveFeedbackRouter:
    
    def __init__(self, model_path: str = "llama-2-13b-chat.Q6_K.gguf", dmn_countdown: int = 60):
        """
        A class that manages the routing of cognitive feedback based on user input and system states.
    
        This class orchestrates various components, including a language learning model (LLM), user input handling,
        and managing different operational modes based on system states like 'sleeping' or 'overwhelmed'.
        """

        self.logger = logging.getLogger(self.__class__.__name__)
        self.logger.info(f"Instantiating {self.__class__.__name__}")
        
        self.stimulus = None
        
        self.overwhelmed = asyncio.Event()
        self.sleeping = asyncio.Event()
        
        self.lock = asyncio.Lock()
        
        self.llm = None
        
        self._model_path = model_path
        self._conversation_handler = None
        
        self.ready_for_input = asyncio.Event()
        self.ready_for_input.set()  # Initially set to ready
        self.conversing = asyncio.Event()
        self.logger.flag(f"'ready_for_input' = {self.ready_for_input.is_set()}, 'conversing' = {self.conversing.is_set()}")

        self._dmn_countdown = dmn_countdown # time between last interaction and entering Default Mode
        
        self.logger.debug("Cognitive Feedback Router instantiated.")
        

    async def _wakeup(self):
        """
        Wakes up the system and initializes the LLM.

        This asynchronous method acquires a lock to ensure thread-safe operations while initializing the LLM.
        It clears the 'sleeping' and 'overwhelmed' states and confirms the wake-up process.
        """
        self.logger.debug("Starting _wakeup() procedure.")    
        async with self.lock:
            self.logger.murmur("Just a second, I'm waking up...")
            self.logger.debug(f"Initializing LLM model from {self._model_path}.")            
            self.llm = LlamaCpp(model_path=self._model_path, 
                                n_ctx=4096, 
                                max_tokens=4000,
                                n_batch=16)
            self.logger.debug(f"LLM model initialized.")                        
            self.sleeping.clear()
            self.overwhelmed.clear()
            self.logger.flag(f"'sleeping' = {self.sleeping.is_set()}, 'conversing' = {self.overwhelmed.is_set()}")
            
    async def _get_user_input(self):
        """
        Continuously captures user input in an asynchronous loop.

        This method waits for the system to be ready for input, then captures and stores user input.
        It clears the 'ready for input' state after capturing the input.
        """

        self.logger.debug(f"Starting user input loop") 
        while True:
            await self.ready_for_input.wait()
            user_input = await asyncio.get_event_loop().run_in_executor(None, input, "Enter something: ")
            
            self.logger.debug(f"User input received: {user_input}.") 
            self.stimulus = user_input
            self.ready_for_input.clear()
            self.logger.flag(f"'ready_for_input' = {self.ready_for_input.is_set()}")

    async def _attention_switch(self):
        """
        Manages the mode of operation based on user input and system states.

        This asynchronous method processes user inputs, manages conversation sessions,
        and handles the 'sleeping' and 'overwhelmed' states of the system.
        """

        self.logger.info(f"Starting infinite attention loop.") 
        while True:
            if not self.sleeping.is_set():
                if self.stimulus:
                    self.logger.debug(f"User input detected: {self.stimulus}") 
                    if not self.conversing.is_set():
                        self.logger.debug(f"Starting new conversation session.")
                        self.conversing.set()
                        self.logger.flag(f"'conversing' = {self.conversing.is_set()}")
                        self._conversation_handler = HumanConversationStimulus(self.llm, self.ready_for_input, self.conversing)
                        asyncio.create_task(self._conversation_handler.start_interaction())
                    self._conversation_handler.pass_stimulus(self.stimulus)
                    self.stimulus = None
                elif self.overwhelmed.is_set():
                    self.logger.info(f"Overwhelmed state detected.") 
                    rem = ReflectiveEvolutionMonitor(llm=self.llm)
                    self.sleeping.set()
                    self.logger.flag(f"'sleeping' = {self.sleeping.is_set()}")
                    await rem.dream()
                    asyncio.create_task(self._wakeup())
                elif not self.conversing.is_set() and not self.overwhelmed.is_set():
                    self.logger.debug(f"No conversation and no new conclusions detected. Preparing to switch to Default Mode.")                     
                    for _ in range(self._dmn_countdown):
                        await asyncio.sleep(1)  # Sleep for 1 second
                        if self.stimulus:
                            self.logger.debug(f"Cancelling Default Mode countdown due to user input detection.")                                                 
                            break  # Exit the loop if new user input is detected
                    else:  # This else clause executes if the loop completes normally (no break)
                        self.logger.debug(f"Entering Default Mode.")                                                                         
                        dmn = DefaultModeNetwork(self.llm, self.overwhelmed)
                        await dmn.run()
                        self.logger.debug(f"Default Mode quit.")                                                                                                 
                else:
                    await asyncio.sleep(1)
            else:
                await asyncio.sleep(1)

    async def run(self):
        """
        Initiates and runs the main functionality of the CognitiveFeedbackRouter.

        This method starts the system by waking it up, initiating user input capture, and entering the mode selection loop.
        """

        self.logger.debug("Cognitive Feedback Router starts.")
        await self._wakeup()
        asyncio.create_task(self._get_user_input())
        await self._attention_switch()

In [45]:
router = CognitiveFeedbackRouter(model_path='llama-2-13b-chat.Q6_K.gguf')
asyncio.run(router.run())

2023-12-13 22:58:01,633 - INFO - CognitiveFeedbackRouter - __init__ - Instantiating CognitiveFeedbackRouter
2023-12-13 22:58:01,635 - DEBUG - CognitiveFeedbackRouter - __init__ - Cognitive Feedback Router instantiated.
2023-12-13 22:58:01,636 - DEBUG - CognitiveFeedbackRouter - run - Cognitive Feedback Router starts.
2023-12-13 22:58:01,636 - DEBUG - CognitiveFeedbackRouter - wakeup - Starting wakeup() procedure.
2023-12-13 22:58:01,637 - MURMUR - CognitiveFeedbackRouter - wakeup - Just a second, I'm waking up...
2023-12-13 22:58:01,638 - DEBUG - CognitiveFeedbackRouter - wakeup - Initializing LLM model from llama-2-13b-chat.Q6_K.gguf.
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 
2023-12-13 22:58:21,046 - DEBUG - CognitiveFeedbackRouter - wakeup - LLM model initialized.
2023-12-13 22:58:21,047 - INFO - CognitiveFeedbackRouter - attention_swit

Enter something:  Hi, how are you today?


2023-12-13 22:58:30,044 - DEBUG - CognitiveFeedbackRouter - get_user_input - User input received: Hi, how are you today?.
2023-12-13 22:58:30,117 - DEBUG - CognitiveFeedbackRouter - attention_switch - Cancelling Default Mode countdown due to user input detection.
2023-12-13 22:58:30,117 - DEBUG - CognitiveFeedbackRouter - attention_switch - User input detected: Hi, how are you today?
2023-12-13 22:58:30,118 - DEBUG - CognitiveFeedbackRouter - attention_switch - Starting new conversation session.
2023-12-13 22:58:30,118 - DEBUG - HumanConversationStimulus - __init__ - Instantiating HumanConversationStimulus with interaction_archive_path: conversations
2023-12-13 22:58:30,119 - DEBUG - HumanConversationStimulus - pass_stimulus - Received Hi, how are you today? from CFR.
2023-12-13 22:58:30,119 - DEBUG - HumanConversationStimulus - start_interaction - Initiated (pre-loop status)
2023-12-13 22:58:30,120 - DEBUG - HumanConversationStimulus - start_interaction - Received Hi, how are you toda

AI: Hello! I'm doing well, thank you for asking. I'm an independent, intelligent entity, so I don't experience emotions or physical sensations like humans do. However, I'm here to assist you with any questions or topics you'd like to discuss. What's on your mind today?


Enter something:  end chat


2023-12-13 22:59:23,529 - DEBUG - CognitiveFeedbackRouter - get_user_input - User input received: end chat.
2023-12-13 22:59:23,632 - DEBUG - CognitiveFeedbackRouter - attention_switch - User input detected: end chat
2023-12-13 22:59:23,632 - DEBUG - HumanConversationStimulus - pass_stimulus - Received end chat from CFR.
2023-12-13 22:59:23,633 - DEBUG - HumanConversationStimulus - start_interaction - Received end chat
2023-12-13 22:59:23,633 - DEBUG - HumanConversationStimulus - _end_interaction - Conversation cleanup started.
2023-12-13 22:59:23,634 - INFO - HumanConversationStimulus - _save_interaction_history - Started conversation saving.
2023-12-13 22:59:23,634 - INFO - HumanConversationStimulus - _summarize_interaction - Conversation summarization started.
2023-12-13 22:59:23,635 - DEBUG - HumanConversationStimulus - _summarize_interaction - Chat history loaded: Human: Hi, how are you today?
AI: Hello! I'm doing well, thank you for asking. I'm an independent, intelligent entity,

KeyboardInterrupt: 